# 多格式支持与数据转换

本notebook演示如何处理不同格式的EEG/LFP数据，包括：
- BrainVision (.vhdr/.eeg/.vmrk)
- FIF (.fif)
- EDF (.edf)
- BDF (.bdf)
- EEGLAB SET (.set)
- Neuroscan CNT (.cnt)

In [10]:
import sys
sys.path.append('/workspace/src')

import numpy as np
import mne
from pathlib import Path

from preprocessing import BIDSDataLoader, BIDSDataSaver, DataValidator

mne.set_log_level('WARNING')

## 1. 自动格式检测

加载器会自动检测文件格式，无需手动指定。

In [4]:
from pathlib import Path
import os

project_root = Path(os.getcwd())
print(f"Current working directory: {project_root}")

# Set BIDS root
bids_root = project_root / 'shared' / 'data' / 'bids_dataset'

# 创建加载器
loader = BIDSDataLoader(bids_root)

# 自动检测并加载（支持.vhdr, .fif, .edf等）
eeg_raw, eeg_metadata = loader.load_eeg_data(
    subject='sub-001',
    session='ses-01',
    task='task-StimOn55HzFull2'
)

print(f"Load EEG data successfully")
print(f"Sampling Rate: {eeg_raw.info['sfreq']} Hz")
print(f"Channel Number: {len(eeg_raw.ch_names)}")

INFO:preprocessing.data_io:检测到文件格式: brainvision (.vhdr)


Current working directory: /workspace


INFO:preprocessing.data_io:✓ 已加载EEG数据: brainvision格式
INFO:preprocessing.data_io:  采样率: 500.0 Hz
INFO:preprocessing.data_io:  通道数: 281
INFO:preprocessing.data_io:  时长: 95.33 秒


Load EEG data successfully
Sampling Rate: 500.0 Hz
Channel Number: 281


## 2. 强制指定格式

如果需要，可以强制指定格式类型。

In [ ]:
# 强制使用BrainVision格式
eeg_raw_bv, _ = loader.load_eeg_data(
    subject='sub-01',
    session='ses-01',
    task='task-rest',
    format_type='brainvision'  # 强制指定
)

# 或强制使用FIF格式
# eeg_raw_fif, _ = loader.load_eeg_data(
#     subject='sub-01',
#     session='ses-01',
#     task='task-rest',
#     format_type='fif'
# )

## 3. 处理BrainVision格式

BrainVision是常用的EEG格式，包含三个文件：
- .vhdr (header)
- .eeg (data)
- .vmrk (markers)

BIDS结构示例：
```
bids_root/
└── sub-01/
    └── ses-01/
        └── eeg/
            ├── sub-01_ses-01_task-rest_eeg.vhdr
            ├── sub-01_ses-01_task-rest_eeg.eeg
            ├── sub-01_ses-01_task-rest_eeg.vmrk
            ├── sub-01_ses-01_task-rest_eeg.json
            └── sub-01_ses-01_task-rest_events.tsv
```

In [7]:
# 使用DataValidator（内部自动使用加载器）
validator = DataValidator(bids_root)

# 会自动检测BrainVision格式
eeg_raw, eeg_meta = validator.load_eeg_data(
    subject='sub-001',
    session='ses-01',
    task='task-StimOn55HzFull2'
)

lfp_raw, lfp_meta = validator.load_lfp_data(
    subject='sub-001',
    session='ses-01',
    task='task-StimOn55HzFull2'
)

INFO:preprocessing.data_io:检测到文件格式: brainvision (.vhdr)
INFO:preprocessing.data_io:✓ 已加载EEG数据: brainvision格式
INFO:preprocessing.data_io:  采样率: 500.0 Hz
INFO:preprocessing.data_io:  通道数: 281
INFO:preprocessing.data_io:  时长: 95.33 秒


FileNotFoundError: 未找到LFP文件: /workspace/shared/data/bids_dataset/sub-001/ses-01/ieeg/sub-001_ses-01_task-StimOn55HzFull2_ieeg.*

## 4. 验证不同格式的数据

无论原始格式是什么，验证流程都是一样的。

In [9]:
# 运行完整验证（自动处理任何支持的格式）
results = validator.run_full_validation(
    subject='sub-01',
    session='ses-01',
    task='task-StimOn55HzFull2'
)

# 生成报告
report = validator.generate_validation_report()
print(report)

INFO:preprocessing.data_validation:============================================================
INFO:preprocessing.data_validation:开始数据验证
INFO:preprocessing.data_validation:============================================================


FileNotFoundError: 未找到EEG文件: /workspace/shared/data/bids_dataset/sub-01/ses-01/eeg/sub-01_ses-01_task-StimOn55HzFull2_eeg.*

## 5. 格式转换

可以在不同格式之间转换数据。

In [ ]:
# 创建保存器
saver = BIDSDataSaver(bids_root, derivatives_name='preprocessing')

# 示例：将BrainVision转为FIF
# 1. 加载BrainVision格式
raw_bv, _ = loader.load_eeg_data(
    'sub-01', 'ses-01', 'task-rest',
    format_type='brainvision'
)

# 2. 保存为FIF格式
saver.save_preprocessed_raw(
    raw_bv,
    subject='sub-01',
    session='ses-01',
    task='task-rest',
    datatype='eeg',
    description='converted',
    format_type='fif'  # 指定输出格式
)

print("格式转换完成！")

## 6. 预处理后保存为不同格式

预处理后可以选择保存格式。

In [ ]:
from preprocessing import EEGCleaner

# 清洗数据
cleaner = EEGCleaner()
eeg_clean = cleaner.apply_eeg_cleaning(eeg_raw)

# 保存为BrainVision格式（推荐用于与其他软件兼容）
saver.save_preprocessed_raw(
    eeg_clean,
    subject='sub-01',
    session='ses-01',
    task='task-rest',
    datatype='eeg',
    description='clean',
    format_type='brainvision'
)

# 或保存为FIF格式（推荐用于MNE-Python）
saver.save_preprocessed_raw(
    eeg_clean,
    subject='sub-01',
    session='ses-01',
    task='task-rest',
    datatype='eeg',
    description='clean-fif',
    format_type='fif'
)

# 或保存为EDF格式（推荐用于临床数据）
saver.save_preprocessed_raw(
    eeg_clean,
    subject='sub-01',
    session='ses-01',
    task='task-rest',
    datatype='eeg',
    description='clean-edf',
    format_type='edf'
)

## 7. 格式选择建议

### BrainVision (.vhdr/.eeg/.vmrk)
**优点：**
- 广泛支持（Brain Products, EEGLAB, FieldTrip等）
- 文本header便于检查
- 保留所有标记信息

**适用于：**
- 需要在多个软件间共享数据
- 临床和研究环境
- 需要保留详细标记信息

**推荐用途：** 原始数据存储、数据共享

---

### FIF (.fif)
**优点：**
- MNE-Python原生格式
- 压缩存储，节省空间
- 读写速度快
- 完整保留MNE信息（投影、处理历史等）

**适用于：**
- MNE-Python工作流
- 中间处理步骤
- Epochs和Evoked数据

**推荐用途：** 预处理流程中的中间文件、epochs保存

---

### EDF (.edf) / BDF (.bdf)
**优点：**
- 临床标准格式
- 广泛的软件支持
- 适合长时程记录

**适用于：**
- 临床数据
- 睡眠研究
- 长时程监测

**推荐用途：** 临床数据存档、睡眠EEG

## 8. 实际工作流建议

### 推荐方案A：BrainVision为主
```python
# 1. 原始数据 → BrainVision格式
# 2. 预处理 → 中间步骤用FIF（快速）
# 3. 最终结果 → BrainVision（兼容性）

# 加载BrainVision
raw, _ = loader.load_eeg_data('sub-01', 'ses-01', 'task-rest')

# 预处理...
raw_clean = process(raw)

# 保存为BrainVision
saver.save_preprocessed_raw(
    raw_clean, 'sub-01', 'ses-01', 'task-rest', 'eeg',
    description='clean', format_type='brainvision'
)
```

### 推荐方案B：FIF为主（MNE工作流）
```python
# 1. 原始数据 → 任意格式
# 2. 转为FIF → 快速处理
# 3. 最终结果 → FIF或BrainVision

# 加载任意格式
raw, _ = loader.load_eeg_data('sub-01', 'ses-01', 'task-rest')

# 预处理...
raw_clean = process(raw)

# 保存为FIF（快速、紧凑）
saver.save_preprocessed_raw(
    raw_clean, 'sub-01', 'ses-01', 'task-rest', 'eeg',
    description='clean', format_type='fif'
)
```

## 9. 完整的预处理示例（混合格式）

In [ ]:
from preprocessing import (
    DataValidator, EEGCleaner, LFPCleaner,
    EEGPreprocessor, BIDSDataSaver
)

# 1. 验证数据（自动检测格式）
validator = DataValidator(bids_root)
results = validator.run_full_validation('sub-01', 'ses-01', 'task-rest')

# 2. 加载数据（可能是BrainVision或其他格式）
eeg_raw, _ = validator.load_eeg_data('sub-01', 'ses-01', 'task-rest')
lfp_raw, _ = validator.load_lfp_data('sub-01', 'ses-01', 'task-rest')

# 3. 预处理
eeg_clean = EEGCleaner().apply_eeg_cleaning(eeg_raw)
lfp_clean = LFPCleaner().apply_lfp_cleaning(lfp_raw)

# 4. 进一步处理
prep = EEGPreprocessor()
bad_chs = prep.detect_bad_channels(eeg_clean)
eeg_clean = prep.interpolate_bad_channels(eeg_clean, bad_chs)
eeg_clean = prep.set_reference(eeg_clean, ref_type='average')

# 5. 保存结果
saver = BIDSDataSaver(bids_root)

# 保存为BrainVision（用于共享）
saver.save_preprocessed_raw(
    eeg_clean, 'sub-01', 'ses-01', 'task-rest', 'eeg',
    description='preprocessed', format_type='brainvision'
)

# 同时保存为FIF（用于后续快速分析）
saver.save_preprocessed_raw(
    eeg_clean, 'sub-01', 'ses-01', 'task-rest', 'eeg',
    description='preprocessed', format_type='fif'
)

print("预处理完成！已保存为多种格式。")

## 总结

- ✅ 自动检测支持的所有格式
- ✅ 统一的API，无需关心底层格式
- ✅ 灵活的格式转换
- ✅ 可以同时保存多种格式
- ✅ 完全兼容原有的预处理流程

**建议：**
- 原始数据使用 **BrainVision** 或 **EDF** （兼容性）
- 预处理过程使用 **FIF** （效率）
- 最终结果同时保存 **BrainVision** 和 **FIF**